In [15]:
!pip install keras-tuner

     |████████████████████████████████| 71kB 2.2MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.2-cp37-none-any.whl size=78938 sha256=5916162c8b74f901152c52b1ee97520f7b088e2bd20dc5f1faf842bde6f4663d
  Stored in directory: /root/.cache/pip/wheels/bb/a1/8a/7c3de0efb3707a1701b36ebbfdbc4e67aedf6d4943a1f463d6
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp37-none-any.whl size=15356 sha256=beaf834c4dcbfda75d7bf6851e610fcd6939a7dc9c7b03266a4d55128d101809
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


In [16]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [17]:
fashion_mnist = keras.datasets.fashion_mnist

In [18]:
(train_images,train_labels),(test_images,test_labels) = fashion_mnist.load_data()

In [19]:
train_images = train_images/255.0
test_images = test_images/255.0

In [20]:
train_images[0].shape

(28, 28)

In [21]:
train_images = train_images.reshape(len(train_images),28,28,1)
test_images = test_images.reshape(len(test_images),28,28,1)

In [28]:
def build_model(hp):  
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])
  
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  return model

In [29]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [30]:
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=5,directory='output',project_name="Mnist Fashion")

In [31]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Trial 5 Complete [00h 01m 22s]
val_accuracy: 0.9016666412353516

Best val_accuracy So Far: 0.9139999747276306
Total elapsed time: 00h 06m 00s
INFO:tensorflow:Oracle triggered exit


In [32]:
model=tuner_search.get_best_models(num_models=1)[0]

In [33]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 80)        800       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 64)        46144     
_________________________________________________________________
flatten (Flatten)            (None, 36864)             0         
_________________________________________________________________
dense (Dense)                (None, 48)                1769520   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                490       
Total params: 1,816,954
Trainable params: 1,816,954
Non-trainable params: 0
_________________________________________________________________
